In [2]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import time
import re
import sys

In [3]:
# Using the Requests Library to make a GET request 
req = requests.get('https://guides.lib.berkeley.edu/information-studies/apis')

# read the content of the server's response
src = req.text

In [4]:
# /html/body/div[5]/div[2]/div[1]/div/div/div/div/div/div/div/p[1]/a[1]

In [5]:
src

'    <!DOCTYPE html>\n<html lang="en">\n<head>\n    <meta http-equiv="X-UA-Compatible" content="IE=Edge"/>\n    <meta http-equiv="Content-Type" content="text/html; charset=utf-8"/>\n    <title>APIs for scholarly resources - Information Studies - Library Guides at UC Berkeley</title><meta name="viewport" content="width=device-width, initial-scale=1.0"><meta name="robots" content="noarchive" /><!-- favicon.twig -->\n<link rel="apple-touch-icon" sizes="180x180" href="//libapps.s3.amazonaws.com/apps/common/favicon/apple-touch-icon.png">\n<link rel="icon" type="image/png" sizes="32x32" href="//libapps.s3.amazonaws.com/apps/common/favicon/favicon-32x32.png">\n<link rel="icon" type="image/png" sizes="16x16" href="//libapps.s3.amazonaws.com/apps/common/favicon/favicon-16x16.png">\n<link rel="manifest" href="//libapps.s3.amazonaws.com/apps/common/favicon/site.webmanifest">\n<link rel="mask-icon" href="//libapps.s3.amazonaws.com/apps/common/favicon/safari-pinned-tab.svg" color="#5bbad5">\n<link 

In [6]:
# Use BeautifulSoup function to parse the response into HTML tree
soup = BeautifulSoup(src, 'lxml')
# returns a soup object, containing all HTML in original doc
print(soup.prettify()[:1000])

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <title>
   APIs for scholarly resources - Information Studies - Library Guides at UC Berkeley
  </title>
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <meta content="noarchive" name="robots"/>
  <!-- favicon.twig -->
  <link href="//libapps.s3.amazonaws.com/apps/common/favicon/apple-touch-icon.png" rel="apple-touch-icon" sizes="180x180"/>
  <link href="//libapps.s3.amazonaws.com/apps/common/favicon/favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/>
  <link href="//libapps.s3.amazonaws.com/apps/common/favicon/favicon-16x16.png" rel="icon" sizes="16x16" type="image/png"/>
  <link href="//libapps.s3.amazonaws.com/apps/common/favicon/site.webmanifest" rel="manifest"/>
  <link color="#5bbad5" href="//libapps.s3.amazonaws.com/apps/common/favicon/safari-pinned-tab.svg" rel="mask-icon

In [15]:
# soup.find_all("a")

In [14]:
soup.select("div.s-lib-box-content", class_="a")

[<div class="s-lib-box-content">
 <div class="clearfix" id="s-lg-content-74224">
 <p>Below is a list of scholarly databases and collections that offer some form of API access, online tools to access data, or raw data downloads.<br/>
 <br/>
 <a href="http://arxiv.org/help/api/index" target="_blank">arXiv API</a><br/>
 API access to Cornell's open-access arXiv e-print repository, used primarily by physics, mathematics and computer science communities to share cutting-edge research.<br/>
 <br/>
 <a href="http://www.biomedcentral.com/about/api" target="_blank">BioMed Central API</a><br/>
 A variety of access points to BioMed Central's corpus of 150,000 peer-reviewed articles, including a <a href="http://www.biomedcentral.com/about/datamining" target="_blank">guide to text-mining the collection</a>.</p>
 <p><a href="https://case.law/">Caselaw Access Project</a><br/>
 Includes all book-published US court decisions. 40,000 volumes with cases from 1658-2018 were scanned from Harvard Law School